<a href="https://colab.research.google.com/github/mralamdari/Robotics-in-Python/blob/main/Robotics_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import scipy
import sympy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#Denavite Hartnzberg
def DH(a, alpha_degree, d, theta_degree):
  theta = np.deg2rad(theta_degree)
  alpha = np.deg2rad(alpha_degree)
  dh_table = np.array([[np.cos(theta), -np.cos(alpha)*np.sin(theta), np.sin(alpha)*np.sin(theta), a*np.cos(theta)],
                      [np.sin(theta), np.cos(alpha)*np.cos(theta), -np.sin(alpha)*np.cos(theta), a*np.sin(theta)],
                      [0 ,np.sin(alpha), np.cos(alpha), d],
                      [0, 0, 0, 1]])
  return np.float16(dh_table)

In [5]:
a1 =DH(0, -90, -999, 0)
a2 =DH(0, -90, -888, 90)
a3 =DH(0, 0, -777, 0)

s=np.dot(a1, a2)
s

array([[   0.,    0.,   -1.,    0.],
       [   0.,   -1.,    0., -888.],
       [  -1.,    0.,    0., -999.],
       [   0.,    0.,    0.,    1.]], dtype=float16)

In [6]:
np.dot(s, a3)

array([[   0.,    0.,   -1.,  777.],
       [   0.,   -1.,    0., -888.],
       [  -1.,    0.,    0., -999.],
       [   0.,    0.,    0.,    1.]], dtype=float16)